In [49]:
# Data processing


import pandas as pd
import numpy as np
import scipy.stats
# Visualization
import seaborn as sns
# Similarity
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
df_ratings=pd.read_csv('/content/drive/Shareddrives/rs assignment1/interactions_train.csv')
# Take a look at the data
df_ratings.head()

user_id  recipe_id        date  rating      u       i
0     2046       4684  2000-02-25     5.0  22095   44367
1     2046        517  2000-02-25     5.0  22095   87844
2     1773       7435  2000-03-13     5.0  24732  138181
3     1773        278  2000-03-13     4.0  24732   93054
4     2046       3431  2000-04-07     5.0  22095  101723

In [52]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698901 entries, 0 to 698900
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    698901 non-null  int64  
 1   recipe_id  698901 non-null  int64  
 2   date       698901 non-null  object 
 3   rating     698901 non-null  float64
 4   u          698901 non-null  int64  
 5   i          698901 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 32.0+ MB


In [53]:
print('The ratings dataset has', df_ratings['user_id'].nunique(), 'unique users')
# Number of recipes
print('The ratings dataset has', df_ratings['recipe_id'].nunique(), 'unique recipes')
# Number of ratings
print('The ratings dataset has', df_ratings['rating'].nunique(), 'unique ratings')
# List of unique ratings
print('The unique ratings are', sorted(df_ratings['rating'].unique()))

The ratings dataset has 25076 unique users
The ratings dataset has 160901 unique recipes
The ratings dataset has 6 unique ratings
The unique ratings are [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]


In [54]:
# Read in data
df_recipes = pd.read_csv('/content/drive/Shareddrives/rs assignment1/RAW_recipes.csv')
# Take a look at the data
df_recipes.head()

name      id  minutes  \
0  arriba   baked winter squash mexican style  137739       55   
1            a bit different  breakfast pizza   31490       30   
2                   all in the kitchen  chili  112140      130   
3                          alouette  potatoes   59389       45   
4          amish  tomato ketchup  for canning   44061      190   

   contributor_id   submitted  \
0           47892  2005-09-16   
1           26278  2002-06-17   
2          196586  2005-02-25   
3           68585  2003-04-14   
4           41706  2002-10-25   

                                                tags  \
0  ['60-minutes-or-less', 'time-to-make', 'course...   
1  ['30-minutes-or-less', 'time-to-make', 'course...   
2  ['time-to-make', 'course', 'preparation', 'mai...   
3  ['60-minutes-or-less', 'time-to-make', 'course...   
4  ['weeknight', 'time-to-make', 'course', 'main-...   

                                    nutrition  n_steps  \
0       [51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]       11   
1   [173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]        9   
2  [269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]        6   
3   [368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]       11   
4   [352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]        5   

                                               steps  \
0  ['make a choice and proceed with recipe', 'dep...   
1  ['preheat oven to 425 degrees f', 'press dough...   
2  ['brown ground beef in large pot', 'add choppe...   
3  ['place potatoes in a large pot of lightly sal...   
4  ['mix all ingredients& boil for 2 1 / 2 hours ...   

                                         description  \
0  autumn is my favorite time of year to cook! th...   
1  this recipe calls for the crust to be prebaked...   
2  this modified version of 'mom's' chili was a h...   
3  this is a super easy, great tasting, make ahea...   
4  my dh's amish mother raised him on this recipe...   

                                         ingredients  n_ingredients  
0  ['winter squash', 'mexican seasoning', 'mixed ...              7  
1  ['prepared pizza crust', 'sausage patty', 'egg...              6  
2  ['ground beef', 'yellow onions', 'diced tomato...             13  
3  ['spreadable cheese with garlic and herbs', 'n...             11  
4  ['tomato juice', 'apple cider vinegar', 'sugar...              8

In [55]:
df_recipes.rename(columns = {'id':'recipe_id'}, inplace = True)

In [56]:
df_new = pd.merge(df_ratings, df_recipes, on='recipe_id', how='inner')
# Take a look at the data
df_new.head()

user_id  recipe_id        date  rating      u      i  \
0     2046       4684  2000-02-25     5.0  22095  44367   
1    12882       4684  2002-03-13     5.0  10399  44367   
2    37758       4684  2002-06-17     5.0   4954  44367   
3    37636       4684  2003-05-08     5.0    831  44367   
4    54697       4684  2003-06-30     5.0   1147  44367   

                                   name  minutes  contributor_id   submitted  \
0  flank steak with lime chipotle sauce       75            1533  1999-11-18   
1  flank steak with lime chipotle sauce       75            1533  1999-11-18   
2  flank steak with lime chipotle sauce       75            1533  1999-11-18   
3  flank steak with lime chipotle sauce       75            1533  1999-11-18   
4  flank steak with lime chipotle sauce       75            1533  1999-11-18   

                                                tags  \
0  ['time-to-make', 'course', 'main-ingredient', ...   
1  ['time-to-make', 'course', 'main-ingredient', ...   
2  ['time-to-make', 'course', 'main-ingredient', ...   
3  ['time-to-make', 'course', 'main-ingredient', ...   
4  ['time-to-make', 'course', 'main-ingredient', ...   

                                      nutrition  n_steps  \
0  [487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]        6   
1  [487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]        6   
2  [487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]        6   
3  [487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]        6   
4  [487.8, 16.0, 283.0, 13.0, 52.0, 20.0, 26.0]        6   

                                               steps description  \
0  ['mix honey with chilies , mustard , lime juic...         yum   
1  ['mix honey with chilies , mustard , lime juic...         yum   
2  ['mix honey with chilies , mustard , lime juic...         yum   
3  ['mix honey with chilies , mustard , lime juic...         yum   
4  ['mix honey with chilies , mustard , lime juic...         yum   

                                         ingredients  n_ingredients  
0  ['flank steaks', 'honey', 'chipotle chiles in ...             10  
1  ['flank steaks', 'honey', 'chipotle chiles in ...             10  
2  ['flank steaks', 'honey', 'chipotle chiles in ...             10  
3  ['flank steaks', 'honey', 'chipotle chiles in ...             10  
4  ['flank steaks', 'honey', 'chipotle chiles in ...             10

In [57]:
df_small = df_new.sample(frac = 0.02)
ratings = df_small[['user_id','recipe_id','rating']]
recipes = df_small[['recipe_id','name']]

In [58]:
from sklearn.preprocessing import LabelEncoder
def ids_encoder(ratings):
    users = sorted(ratings['user_id'].unique())
    items = sorted(ratings['recipe_id'].unique())
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()
    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)
    # encode userids and itemids
    ratings.user_id = uencoder.transform(ratings.user_id.tolist())
    ratings.recipe_id = iencoder.transform(ratings.recipe_id.tolist())

    return ratings, uencoder, iencoder

In [59]:
# create the encoder
ratings, uencoder, iencoder = ids_encoder(ratings)
#new_ratings['rating'] = new_ratings['rating'].astype('int64')
ratings.shape
ratings.head()


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


user_id  recipe_id  rating
568723     4935       8888     5.0
554742     5080       8057     4.0
617135     1591      10207     5.0
523769     4419       8728     5.0
49356      4280        933     4.0

In [60]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from scipy.spatial.distance import correlation

In [61]:
def ratings_matrix(ratings): 
    return csr_matrix(pd.crosstab(ratings.user_id, ratings.recipe_id, ratings.rating, aggfunc=sum).fillna(0).values)    

In [62]:
R = ratings_matrix(ratings)

In [63]:
def create_model(rating_matrix, metric):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=6, algorithm='brute')
    model.fit(rating_matrix)    
    return model

In [64]:
def nearest_neighbors(rating_matrix, model):
    """    
    :param rating_matrix : rating matrix of shape (nb_users, nb_items)
    :param model : nearest neighbors model    
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

In [65]:
model = create_model(rating_matrix=R, metric='cosine') # we can also use the 'euclidian' distance
similarities, neighbors = nearest_neighbors(R, model)

In [66]:
def find_candidate_items(userid):
    """
    Find candidate items for an active user
    
    :param userid : active user
    :param neighbors : users similar to the active user        
    :return candidates : top 30 of candidate items
    """
    try:
      user_neighbors = neighbors[userid]
    except IndexError:
      print('There is no item in the array at index 0')
    
    activities = ratings.loc[ratings.user_id.isin(user_neighbors)]
     # sort items in decreasing order of frequency
    frequency = activities.groupby('recipe_id')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.recipe_id
    active_items = ratings.loc[ratings.user_id == userid].recipe_id.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]
        
    return candidates

In [67]:
# mean ratings for each user
mean = ratings.groupby(by='user_id', as_index=False)['rating'].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='user_id')

# normalized ratings for each items
mean_ratings['norm_rating'] = mean_ratings['rating'] - mean_ratings['rating_mean']

mean = mean.to_numpy()[:, 1]

In [68]:
np_ratings = mean_ratings.to_numpy()

In [69]:
def predict(userid, itemid):
    """
    predict what score userid would have given to itemid.
    
    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction
        
    :return
        - r_hat : predicted rating of user userid on item itemid
    """
    user_similarities = similarities[userid]
    user_neighbors = neighbors[userid]
    # get mean rating of user userid
    user_mean = mean[userid]

     # find users who rated item 'itemid'
    iratings = np_ratings[np_ratings[:, 1].astype('int') == itemid]
    
    # find similar users to 'userid' who rated item 'itemid'
    suri = iratings[np.isin(iratings[:, 0], user_neighbors)]
        # similar users who rated current item (surci)
    normalized_ratings = suri[:,4]
    indexes = [np.where(user_neighbors == uid)[0][0] for uid in suri[:, 0].astype('int')]
    sims = user_similarities[indexes]
    
    num = np.dot(normalized_ratings, sims)
    den = np.sum(np.abs(sims))
    
    if num == 0 or den == 0:
        return user_mean
    
    r_hat = user_mean + np.dot(normalized_ratings, sims) / np.sum(np.abs(sims))
    
    return r_hat


In [70]:
def user2userPredictions(userid, pred_path):
    """
    Make rating prediction for the active user on each candidate item and save in file prediction.csv
    
    :param
        - userid : id of the active user
        - pred_path : where to save predictions
    """    
    # find candidate items for the active user
    candidates = find_candidate_items(userid)
    list = []
    # loop over candidates items to make predictions
    for itemid in candidates:
        
        # prediction for userid on itemid
        r_hat = predict(userid, itemid)
        
       
      # save predictions
        with open(pred_path, 'a+') as file:
            line = '{},{},{}\n'.format(userid, itemid, r_hat)
            file.write(line)
         

In [71]:
import sys
import os
def user2userCF():
    """
    Make predictions for each user in the database.    
    """
    # get list of users in the database
    users = ratings.user_id.unique()
    print(len(users))
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
    
    saved_predictions = 'predictions.csv'    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    for count, userid in enumerate(users):        
        # make rating predictions for the current user
        user2userPredictions(userid, saved_predictions)
        _progress(count)

In [72]:
user2userCF()

5567
Rating predictions. Progress status : 100.0%

In [73]:
def user2userRecommendation(userid):
    """
    """
    # encode the userid
    uid = userid
    saved_predictions = 'predictions.csv'
    
    predictions = pd.read_csv(saved_predictions, sep=',', names=['user_id', 'recipe_id', 'predicted_rating'])
    predictions = predictions[predictions.user_id==uid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)
    
    List.user_id = List.user_id.tolist()
    List.recipe_id = List.recipe_id.tolist()
    
    List = pd.merge(List, recipes, on='recipe_id', how='inner')
    
    return List

In [79]:
test_user = ratings['user_id'].iloc[1]
user2userRecommendation(test_user)# pass user id here

user_id  recipe_id  predicted_rating                    name
0     5080       7844          4.285714        orange marmalade
1     5080       7553          4.000000  strawberry rhubarb pie
2     5080       7553          4.000000  strawberry rhubarb pie

In [80]:
examples = ratings[['user_id', 'recipe_id']].values
labels = ratings['rating'].values

In [81]:
 # split data into train and test sets
from sklearn.model_selection import train_test_split 
train_examples, test_examples, train_labels, test_labels = train_test_split(
        examples, 
        labels, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True)
# transform train and test examples to their corresponding one-hot representations
train_users = train_examples[:, 0]
test_users = test_examples[:, 0]

train_items = train_examples[:, 1]
test_items = test_examples[:, 1]

    # Final training and test set
x_train = np.array(list(zip(train_users, train_items)))
x_test = np.array(list(zip(test_users, test_items)))

y_train = train_labels
y_test = test_labels

In [82]:
def evaluate(x_test, y_test):
        print('Evaluate the model on {} test data ...'.format(x_test.shape[0]))
        preds = list(predict(u, i) for (u, i) in x_test)
        mae = np.sum(np.absolute(y_test - np.array(preds))) / x_test.shape[0]
        print('\nMAE :', mae)
        return mae

In [83]:
evaluate(x_test, y_test)

Evaluate the model on 2796 test data ...

MAE : 0.34200530190901945


0.34200530190901945